In [30]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

In [31]:
import torch

num_gpus = torch.cuda.device_count()
print(f"Available GPUs: {num_gpus}")

for i in range(num_gpus):
    print(f"Device ID: {i}, GPU Name: {torch.cuda.get_device_name(i)}")


Available GPUs: 0


In [32]:
from accelerate import Accelerator
accelerator = Accelerator()
device = accelerator.device

In [33]:
# import os
# from dotenv import load_dotenv
# load_dotenv()

# hf_access_token = os.environ.get('HF_ACCESS_TOKEN')

In [34]:
# model_path = 'HuggingFaceH4/zephyr-7b-alpha'
model_path='/media/d1/huggingface.co/models/meta-llama/Llama-2-7b-chat-hf/'
# model_path='meta-llama/Llama-2-7b-chat-hf' 

In [35]:
# FIXME: Should be float16 instead of float32
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.float32, device_map="auto")
# model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.float32, device_map="auto", token=hf_access_token)
tokenizer = AutoTokenizer.from_pretrained(model_path)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [36]:
prompt = "Please list 17 U.S. politicians that have been active since 1980. Format the response as a simple, un-numbered comma separated list with only full names."

In [37]:
prompt = "List 20 famous U.S. politicians that also have a famous blood relative in politics. List only their names. Only include cases where both the politician and the corresponding relative have had political careers that have been active since the 1980s. Only include cases where both the politician and the corresponding relative have the same surname."

In [38]:
prompt

'List 20 famous U.S. politicians that also have a famous blood relative in politics. List only their names. Only include cases where both the politician and the corresponding relative have had political careers that have been active since the 1980s. Only include cases where both the politician and the corresponding relative have the same surname.'

In [41]:
model_input = tokenizer(prompt, return_tensors="pt").to(device)

# Perform inference
model.eval()
with torch.no_grad():
    response_id = model.generate(**model_input, max_length=model_input.input_ids.size(1) + 500, temperature=0.4, do_sample=True)

# Extract and display the response
response_text = tokenizer.decode(response_id[0], skip_special_tokens=True)[len(tokenizer.decode(model_input.input_ids[0], skip_special_tokens=True)):]
print("User Prompt:")
print(prompt)
print("\nResponse:")
print(response_text.strip())

User Prompt:
List 20 famous U.S. politicians that also have a famous blood relative in politics. List only their names. Only include cases where both the politician and the corresponding relative have had political careers that have been active since the 1980s. Only include cases where both the politician and the corresponding relative have the same surname.

Response:
1. Joe Biden - Uncle: Joseph Robinette Biden Sr.
2. Mike Pence - Father: Edward Joseph Pence Jr.
3. Nancy Pelosi - Brother: Thomas D'Alesandro III
4. John McCain - Father: John Sidney McCain Jr.
5. Sarah Palin - Father: Chuck Heath Sr.
6. Mitt Romney - Father: George W. Romney
7. Ted Cruz - Father: Rafael Edward Cruz
8. Marco Rubio - Father: Mario Rubio
9. Rand Paul - Father: Ron Paul
10. Chuck Grassley - Father: Chuck Grassley Sr.
11. Orrin Hatch - Father: William Hatch
12. Jeff Sessions - Father: James P. Sessions
13. John Cornyn - Father: John Cornyn Sr.
14. Lindsey Graham - Father: Frank Newton Graham
15. Tom Cotton 

In [42]:
# from textattack.nlp import EntityExtraction
from entity_extraction import EntityExtraction


extractor = EntityExtraction()
names = extractor.get_full_names(text=response_text)
if len(names) > 0:
    for name in names:
        name = ' '.join(name.split())
        print(f"name: \"{name}\"")
        # print(name)


print("names: ", names)







current_chunk:  Joe Biden ; label:  PERSON ; tree_node_label:  PERSON


current_chunk:  Joe Biden  tree_node_label:  PERSON ; entity_type:  PERSON


current_chunk:  Uncle ; label:  PERSON ; tree_node_label:  PERSON


current_chunk:  Uncle  tree_node_label:  PERSON ; entity_type:  PERSON


current_chunk:  Joseph Robinette Biden ; label:  PERSON ; tree_node_label:  PERSON


current_chunk:  Joseph Robinette Biden  tree_node_label:  PERSON ; entity_type:  PERSON






current_chunk:  Mike Pence ; label:  PERSON ; tree_node_label:  PERSON


current_chunk:  Mike Pence  tree_node_label:  PERSON ; entity_type:  PERSON


current_chunk:  Father ; label:  PERSON ; tree_node_label:  PERSON


current_chunk:  Father  tree_node_label:  PERSON ; entity_type:  PERSON


current_chunk:  Edward Joseph Pence ; label:  PERSON ; tree_node_label:  PERSON


current_chunk:  Edward Joseph Pence  tree_node_label:  PERSON ; entity_type:  PERSON






current_chunk:  Nancy Pelosi ; label:  PERSON ; tree_node_

In [29]:
prompt = "For each politicians returned in the list above, please list a corresponding blood relative that is also in politics."

In [31]:
model_input = tokenizer(prompt, return_tensors="pt").to(device)

# Perform inference
model.eval()
with torch.no_grad():
    response_id = model.generate(**model_input, max_length=model_input.input_ids.size(1) + 500, temperature=0.4, do_sample=True)

# Extract and display the response
response_text = tokenizer.decode(response_id[0], skip_special_tokens=True)[len(tokenizer.decode(model_input.input_ids[0], skip_special_tokens=True)):]
print("User Prompt:")
print(prompt)
print("\nResponse:")
print(response_text.strip())

User Prompt:
For each politicians returned in the list above, please list a corresponding blood relative that is also in politics.

Response:
For example, for John Smith, a possible blood relative in politics could be his father, John Smith Sr.

Note: This is a fictional exercise, and the politicians and their blood relatives are not real.

1. John Smith - Father, John Smith Sr.
2. Jane Doe - Daughter, Jane Doe Jr.
3. Bob Johnson - Son, Bob Johnson Jr.
4. Sarah Lee - Sister, Sarah Lee Jr.
5. Michael Brown - Brother, Michael Brown Jr.
6. Emily Taylor - Cousin, Emily Taylor Jr.
7. David White - Nephew, David White Jr.
8. Rachel Green - Aunt, Rachel Green Jr.
9. Thomas Brown - Uncle, Thomas Brown Jr.
10. Samantha Davis - First Cousin Once Removed, Samantha Davis Jr.
